# Train test split KVLCC2 HSVA with propeller model
<cite id="hzn48"><a href="#zotero|7505983/U2L55C9Q">(Luo et al., 2016)</a></cite>

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (10,10)
from src.symbols import *

# Read configs:
conf_path = os.path.join("../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression, create_full_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from src.models.vmm import VMM
from src.parameters import df_parameters
from src.models.propeller import fit, predict, predictor

In [ ]:
ship="kvlcc2_hsva"
vmm_name = "vmm_martins_simple"
#vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)

ship_data = catalog.load(f"{ship}.ship_data")

train = "HSVA_CPMC_KVLCC2_Z_25_05"
df_train = catalog.load(f'{ship}.updated.{train}.data_ek_smooth')

test = "HSVA_CPMC_KVLCC2_Z_35_05"
df_test = catalog.load(f'{ship}.updated.{test}.data_ek_smooth')

#delta_start = np.deg2rad(10.5)
#
#for data in [df_train, df_test]:
#    index_start = (data['delta'].diff().abs() > 0).idxmax()
#    start = data.loc[0:index_start] 
#    start['delta'] = delta_start

#index_start = (df_train['delta'].diff().abs() > 0).idxmax()
#df_train = df_train.loc[index_start:].copy()
#df_train.index = df_train.index - df_train.index[0]

index_start = (df_test['delta'].diff().abs() > 0).idxmax()
df_test = df_test.loc[index_start:].copy()
df_test.index = df_test.index - df_test.index[0]


added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")

In [ ]:
data = df_train
data_with_force = predict_force(data=data, added_masses=added_masses, ship_parameters=ship_data, vmm=vmm)


In [ ]:
vmm.X_eq

In [ ]:
vmm.Y_eq

In [ ]:
vmm.N_eq

In [ ]:
regression, parameters = fit_motions(data=data_with_force, added_masses=added_masses, ship_data=ship_data, vmm=vmm, exclude_parameters=exclude_parameters)

In [ ]:
model = create_model_from_motion_regression(regression=regression)

## Thrust model

In [ ]:
propeller_coefficients = catalog.load("kvlcc2.propeller_coefficients")

In [ ]:
add_constant = False
model_pos, model_neg = fit(data=data, ship_data=ship_data, propeller_coefficients=propeller_coefficients, add_constant=add_constant) 
data_predicted_thrust = predict(model_pos=model_pos, model_neg=model_neg, data=data_with_force, propeller_coefficients=propeller_coefficients, 
                                ship_data=ship_data, add_constant=add_constant)

regression_predicted_thrust, parameters = fit_motions(data=data_predicted_thrust, added_masses=added_masses, 
                                                      ship_data=ship_data, vmm=vmm, exclude_parameters=exclude_parameters)

In [ ]:
model_predicted_thrust = create_full_model_from_motion_regression(regression=regression_predicted_thrust, 
                                                                  model_pos=model_pos, 
                                                                  model_neg=model_neg, 
                                                                  propeller_coefficients=propeller_coefficients)

In [ ]:
df_ = df_test.copy()
df_predicted_thrust = predict(model_pos=model_pos, model_neg=model_neg, data=df_test, propeller_coefficients=propeller_coefficients, 
                                ship_data=ship_data, add_constant=add_constant)

#df_.iloc[0]['r'] = 0
#df_.iloc[0]['r'] = -0.03
#df_['delta']+=np.deg2rad(1.5)
    
result = model.simulate(df_=df_)
result_predicted_thrust = model_predicted_thrust.simulate(df_=df_.drop(columns=['thrust']))

df_result = result.result.copy()
df_result_predicted_thrust = result_predicted_thrust.result.copy()
df_result_predicted_thrust['thrust'] = predict(model_pos=model_pos, 
                                               model_neg=model_neg, 
                                               data=df_result_predicted_thrust, 
                                               propeller_coefficients=propeller_coefficients, 
                                               ship_data=ship_data)['thrust']

for df in [df_test, df_result, df_result_predicted_thrust]:
    df['psi_deg'] = np.rad2deg(df['psi'])
    df['delta_deg'] = np.rad2deg(df['delta'])

dataframes = {
    'Experiment' : df_test,
    'Hull + Rudder prediction' : df_result,
    'Full prediction' : df_result_predicted_thrust,
    
}

styles = {
    'Experiment' : {'style':'r--'},
    'Hull + Rudder prediction' : {'style':'b--'},
    'Full prediction' : {'style':'k-'},
}




In [ ]:
track_plots(dataframes, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles);

In [ ]:
#plot(dataframes=dataframes, keys=['delta_deg','psi_deg','y0','u','v','r'], ncols=1, styles=styles, time_window=[0,2]);
plot(dataframes=dataframes, keys=['delta_deg','psi_deg','thrust','y0','u','v','r'], ncols=1, styles=styles);

In [ ]:
u0 = data['u'].iloc[0]
rev = data['rev'].iloc[0]
result2 = model_predicted_thrust.zigzag(u0=u0, rev=rev, angle=35, heading_dev=5)
result2.include_accelerations=False
df_result2 = result2.result.copy()
for df in [df_result2]:
    df['psi_deg'] = np.rad2deg(df['psi'])
    df['delta_deg'] = np.rad2deg(df['delta'])

In [ ]:
df_result2['thrust'] = predict(model_pos=model_pos, model_neg=model_neg, data=df_result2, propeller_coefficients=propeller_coefficients, ship_data=ship_data)['thrust']

In [ ]:
track_plots({'sim':result2.result}, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles);

In [ ]:
plot({'sim':df_result2}, keys=['delta_deg','psi_deg','thrust','y0','u','v','r'], ncols=1, time_window=[0,10]);

In [ ]:
u0 = data['u'].iloc[0]
rev = data['rev'].iloc[0]
result2 = model_predicted_thrust.turning_circle(u0=u0, rev=rev, angle=35)
result2.include_accelerations=False
df_result2 = result2.result.copy()

In [ ]:
df_result2['thrust'] = predict(model_pos=model_pos, model_neg=model_neg, data=df_result2, propeller_coefficients=propeller_coefficients, ship_data=ship_data)['thrust']

In [ ]:
track_plots({'sim':result2.result}, lpp=ship_data['L'], beam=ship_data['B'],  styles=styles);

In [ ]:
plot({'sim':df_result2}, keys=['delta','psi','thrust','y0','u','v','r'], ncols=1,);